## First loading in modules

In [1]:
from masking_py3 import maskmaker
import numpy as np
import matplotlib.pyplot as plt
from psana import * 

## Next the tunables

In [11]:
run = 22
Nevents = 100000.
datadir = '/sdf/data/lcls/ds/cxi/cxil1035023/scratch/lhuang3/'#'/sdf/data/lcls/ds/cxi/cxix1000021/scratch/nag1647/Masks/'
detector = 'front' 
#filename = datadir+'path/%dfile_%d.h5' %(run, Nevents/1000)
#filename = '/sdf/data/lcls/ds/cxi/cxil1035023/scratch/lhuang3/run95_100_stats.h5'#datadir+'run%d_%d_stats.h5' %(run, Nevents/1000)
whichmask = 'dark' #'dark' or 'xray'
# whichmask = 'xray'

if whichmask == 'xray':
	oldmask = np.load(datadir+'run%d_mask_dark.npy' %run)
   # oldmask = np.load(datadir+'Mask_Jungfrau_T_Edge_Corrected.npy')
if whichmask == 'dark':
	oldmask = np.load(datadir+'Mask_Jungfrau_T_Edge_Corrected.npy') #mask of just edge & unbonded pixels
    
#print(np.sum(oldmask))  
#for i in range(2,4):
#    oldmask[i,252:257,:] = 0
#    oldmask[i,:,766:769] = 0
#for i in range(4,6):
#    oldmask[i,252:257,:] = 0
#    oldmask[i,:,254:257] = 0
#for i in range(0,8):
#    oldmask[i,0,:] = 0
#    oldmask[i,511,:] = 0
#    oldmask[i,:,0] = 0
#    oldmask[i,:,1023] = 0

#np.save('Mask_Jungfrau_T_Edge_Corrected.npy', oldmask)


In [12]:
print(filename)

/sdf/data/lcls/ds/cxi/cxil1006021/scratch/lhuang3/run95_100_stats.h5


## Make the Mask

In [8]:
import h5py
hf = h5py.File(filename, 'r')
print(hf.keys())
print(np.sum(hf['xray_shots'][0]))

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/sdf/data/lcls/ds/cxi/cxil1006021/scratch/lhuang3/run95_100_stats.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
#maskmaker makes the mask, starting from the 'oldmask' input
#from masking_py3 import maskmaker
mm = maskmaker('cxil1006021', run, detector, filename, oldmask)

xray_image = np.copy(mm.xray)
dark_image = np.copy(mm.dark)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/sdf/data/lcls/ds/cxi/cxil1006021/scratch/lhuang3/run108_100_stats.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Plot Average Detector Signal

In [ ]:
print('average detector signal')
print(np.average(mm.intensity))
plt.hist(mm.intensity)
plt.title('jungfrau intensity')
plt.ylabel('shot counts')
plt.xlabel('intensity per shot')
plt.show()

## Find Upper and Lower Bounds

In [ ]:
###### print 'use the histogram to find lower and upper bounds'
if whichmask == 'xray':
	plt.hist(xray_image.flatten(), bins=552, range = (-0.25, 0.25))
	plt.title('xray-on image')
if whichmask == 'dark':
	plt.hist(np.nan_to_num(dark_image.flatten()), bins = 100)
	plt.title('xray-off image run %d' % run)
plt.ylabel('pixel counts')
plt.xlabel('ADU per pixel')
plt.show()
#plt.savefig('run%d_dark.png' % run)

print('input the lower bound:')
lb = float(input())
print('input the upper bound:')
ub = float(input())

assert ub > lb

## Make new mask

In [ ]:
name = datadir+'run%d_mask_%s.npy' %(run, whichmask)
newmask = mm(whichmask, lowerbound=lb, upperbound=ub, tolerance=2.5, name=name)
print(newmask.sum())
print(newmask.sum()/(8*512*1024))

## Show Xray On Jungfrau Image

In [ ]:
img = mm.det.image(mm.evt0, xray_image)
#plt.imshow(img, vmin=lb, vmax=ub, cmap = 'jet')
plt.imshow(img, vmin = -2, vmax = 2, cmap = 'jet')
plt.colorbar()
plt.title('xray-on jungfrau image')
plt.show()

## Show Dark Image

In [ ]:
img = mm.det.image(mm.evt0, dark_image)
plt.imshow(img, vmin=-2, vmax=2, cmap ='jet')
plt.title('dark jungfrau image')
plt.colorbar()
plt.show()

## Show Masked Image

In [ ]:
img = mm.det.image(mm.evt0, newmask* xray_image)
#plt.imshow(img, vmin=lb, vmax=ub)
plt.imshow(img, vmin=-2, vmax=2, cmap = 'jet')
plt.title('dark jungfrau image')
plt.colorbar()
plt.title('masked jungfrau')
plt.show()

In [ ]:
img = mm.det.image(mm.evt0, newmask)
plt.imshow(img, vmin=0, vmax=1.3, cmap = 'jet')
plt.title('Basic Mask')
plt.colorbar()
plt.show()
#plt.savefig('analysis_mask.png')

## Let's find the 'Bad T' sections and remove them

In [ ]:
Jungfrau_Test = newmask
sum_value_array_512 = np.zeros(512)
sum_value_array_1024 = np.zeros(1024)
for i in range(0,512):
    sum_value = np.sum(Jungfrau_Test[4,i,:])
    sum_value_array_512[i] = sum_value
for i in range(0, 1024):
    sum_value = np.sum(Jungfrau_Test[4,:,i])
    sum_value_array_1024[i] = sum_value
    
print(np.min(sum_value_array_512))
print(np.where(sum_value_array_512 == np.min(sum_value_array_512)))
print(np.min(sum_value_array_1024))
print(np.where(sum_value_array_1024 == np.min(sum_value_array_1024)))
print(sum_value_array_512[245:265]) #It looks like columns 252-257 should be omitted
print(sum_value_array_1024[245:265]) #It looks like rows 254-257 should be omitted in modules 4 and 5

reg_array = np.arange(1024)
flip_array = np.flip(reg_array)
print(flip_array[254]) 
print(flip_array[257]) #The results from 'flip array' indicate rows 766-769 should be omitted in modules 2 and 3

    